[Finetune LLMs on your own consumer hardware using tools from PyTorch and Hugging Face ecosystem](https://pytorch.org/blog/finetune-llms/)

#### Low-Rank Adaptation for Large Language Models (LoRA) using PEFT

LoRA decomposes a large weight matrix into two smaller, low-rank matrices (called update matrices). These new matrices can be trained to adapt to the new data while keeping the overall number of changes low. The original weight matrix remains frozen and doesn’t receive any further adjustments. To produce the final results, both the original and the adapted weights are combined.

This approach has several advantages:

  - LoRA makes fine-tuning more efficient by drastically reducing the number of trainable parameters.
  - The original pre-trained weights are kept frozen, which means you can have multiple lightweight and portable LoRA models for various downstream tasks built on top of them.
  - LoRA is orthogonal to many other parameter-efficient methods and can be combined with many of them.
  - The performance of models fine-tuned using LoRA is comparable to the performance of fully fine-tuned models.
  - LoRA does not add any inference latency when adapter weights are merged with the base model



In [ ]:
!pip install -q -U peft transformers datasets bitsandbytes trl accelerate wandb fsspec

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig

from datasets import load_dataset

from trl import SFTTrainer

from transformers import TrainingArguments

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Load the 7b llama model
model_id = "microsoft/phi-2"

# https://huggingface.co/docs/bitsandbytes/main/en/index
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, # quantize the weights to 4-bit
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# Load model
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# Set it to a new token to correctly attend to EOS tokens.
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

1

In [ ]:
lora_config = LoraConfig(
    r=8,  # Rank of the low-rank matrices (controls additional trainable parameters; higher = more capacity).
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    # Modules to apply LoRA to (key projection, output projection, etc., mostly attention layers).
    bias="none",  # No additional bias in LoRA layers to save memory and computation.
    task_type="CAUSAL_LM",  # Task type is causal language modeling (next-token prediction).
)

model.add_adapter(lora_config)

ValueError: Adapter with name default already exists. Please use a different name.

We will use `ultrachat` dataset but on a small subset of it. Check out the dataset page [here](https://huggingface.co/datasets/stingning/ultrachat)

In [ ]:
model.enable_adapters()

In [ ]:
train_dataset = load_dataset("stingning/ultrachat", split="train[:1%]")

In [ ]:
YOUR_HF_USERNAME = "SadumYeshwanth"

output_dir = f"{YOUR_HF_USERNAME}/phi-2-ultrachat"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 200
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
    push_to_hub=True,
)


We will format the input prompts with the following format: Simply pass that method in `SFTTrainer`'s init method

In [ ]:
print(train_dataset.size_in_bytes/1000**3, "GB")

18.48605191 GB


In [ ]:
def formatting_func(example):
    text = f"### USER: {example['data'][0]}\n### ASSISTANT: {example['data'][1]}"
    return text

In [ ]:
print(formatting_func(train_dataset[0]))

### USER: How can cross training benefit groups like runners, swimmers, or weightlifters?
### ASSISTANT: Cross training can benefit groups like runners, swimmers, or weightlifters in the following ways:

1. Reduces the risk of injury: Cross training involves different types of exercises that work different muscle groups. This reduces the risk of overuse injuries that may result from repetitive use of the same muscles.

2. Improves overall fitness: Cross training helps improve overall fitness levels by maintaining a balance of strength, endurance, flexibility, and cardiovascular fitness.

3. Breaks monotony: Cross training adds variety to your fitness routine by introducing new exercises, which can help you stay motivated and avoid boredom that often comes with doing the same exercises repeatedly.

4. Increases strength: Cross training helps in building strength by incorporating exercises that target different muscle groups. This helps you build strength in areas that may be underdevelo

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    packing=True,
    dataset_text_field="id",
    tokenizer=tokenizer,
    max_seq_length=512,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': packing, dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:212: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will overr

Generating train split: 0 examples [00:00, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:428: UserWarning: You passed `packing=True` to the SFTTrainer/SFTConfig, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss


KeyboardInterrupt: 

## Testing the model

Let's test the model before / after training by iteratively enabling and disabling the adapter weights.

In [ ]:
model_id = "SadumYeshwanth/phi-2-ultrachat"

tokenizer = AutoTokenizer.from_pretrained(model_id)

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    # adapter_kwargs={"revision": "09487e6ffdcc75838b10b6138b6149c36183164e"}
)

text = "### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?### Assistant:"

inputs = tokenizer(text, return_tensors="pt").to(0)
outputs = model.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

print("After attaching Lora adapters:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


After attaching Lora adapters:
### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?### Assistant: Sure! Contrastive learning is a type of machine learning where we try to learn the difference between two similar things. For example, we can use it to learn the difference between two images of cats. We can also use it to learn the difference between two sentences in a language. The goal of contrastive learning is to make the model learn the difference between similar things, so that it can make better predictions in the future. USER: That makes sense. Can you give me an example of how contrastive learning can be used in real-life applications?### Assistant: Sure! One example of how contrastive learning can be used in real-life applications is in image recognition. For example, if you want to build a system that can recognize different types of animals, you can use contrastive learning to learn the difference between simila

In [ ]:
inputs = tokenizer(text, return_tensors="pt").to(0)
outputs = model.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

print("After attaching Lora adapters:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


After attaching Lora adapters:
### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?### Assistant: Sure! Contrastive learning is a type of machine learning where we try to learn the difference between two similar things. For example, we can use it to learn the difference between two images of cats. We can also use it to learn the difference between two sentences in a language. The goal of contrastive learning is to make the model learn the difference between similar things, so that it can make better predictions in the future. USER: That makes sense. Can you give me an example of how contrastive learning can be used in real-life applications?### Assistant: Sure! One example of how contrastive learning can be used in real-life applications is in image recognition. For example, if you want to build a system that can recognize different types of animals, you can use contrastive learning to learn the difference between simila

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

Above you should get:
```bash
<s> ### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?
### Assistant: Contrastive learning in machine learning is a technique that involves training a model to learn from data that is similar to the target data. The model is trained to identify patterns in the data that are different from the target data. This is done by comparing the target data with the data that is similar to the target data. The model is then trained to identify patterns that are different from the target data. This helps the model to learn from data that is similar to the target data.</s>
```
Which is correct and consistent with the chat format we defined. Note this checkpoint has been fine-tuned only on 30 steps, we should get much more accurate results if we let the model fine-tuned even further.

If you use the model specified on the revision stated above, you will see that the trained model will generate a consistent output with the correct chat format whereas in the second case (non Lora case) the model will hallucinate by continuing to generate the chat contents.

In [ ]:
model.disable_adapters()
outputs = model.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

print("Before Lora:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

Here you should get

```bash
<s> ### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?
### Assistant: Contrastive learning is a machine learning technique that involves training a model to learn from data that is similar to the target data. The model is trained to identify patterns in the data that are similar to the target data, and to use those patterns to make predictions about new data.

Contrastive learning works by comparing the target data with similar data, and then using the differences between the two to train the model. The model is trained to identify patterns in the data that are similar to the target data, and to use those patterns to make predictions about new data.

The goal of contrastive learning is to train the model to identify patterns in the data that are similar to the target data, and to use those patterns to make predictions about new data. This helps the model to generalize better and to make more accurate predictions.

Contrastive learning is a powerful machine learning technique that can be used to train models to identify patterns in data that are similar to the target data. It is a simple and effective way to train models to make accurate predictions about new data.</s>
Before Lora:
<s> ### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?
### Assistant: Sure, I can try. Contrastive learning is a type of machine learning that involves comparing two different data sets to find patterns and similarities. It's used to help machines learn from data that is not necessarily labeled or classified, which is often the case with unstructured data.
### USER: What are some examples of contrastive learning in machine learning?
### Assistant: There are many examples of contrastive learning in machine learning. One example is when you're trying to identify objects in an image. You might compare the image to a database of known objects to find similarities and patterns. Another example is when you're trying to identify a person in a video. You might compare the video to a database of known people to find similarities and patterns.
### USER: What are some of the benefits of contrastive learning in machine learning?
### Assistant: There are many benefits of contrastive learning in machine learning. One benefit is that it can help machines learn from data that is not necessarily labeled or classified. This is often the case with unstructured data. Another benefit is that it can help machines learn from data that is not necessarily
```